In [1]:
%%writefile MyCSVDAG.py

import datetime as dt
from datetime import timedelta
from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator
import pandas as pd
 
def CSVToJson():
    df=pd.read_csv('/opt/airflow/dags/data/data.csv')
    for i,r in df.iterrows():
        print(r['name'])
 
    df.to_json('/opt/airflow/dags/data/fromAirflow.json',orient='records')
 
 
default_args = {
 
    'owner': 'ibrahim1',
 
    'start_date': dt.datetime(2020, 3, 18),
 
    'retries': 1,
 
    'retry_delay': dt.timedelta(minutes=1),
}
 
with DAG('MyCSVDAG',
 
         default_args=default_args,
         schedule_interval=timedelta(minutes=1),  
        catchup=False,     
         ) as dag:
 
    print_starting = BashOperator(task_id='starting',
 
                     bash_command='echo "I am reading the       CSV now....."')
 
 
 
    CSVJson = PythonOperator(task_id='convertCSVtoJson',
 
                             python_callable=CSVToJson)
 
 
print_starting.set_downstream(CSVJson)
# or 
# CSVJson.set_upstream(print_starting)
 

print_starting >>  CSVJson
 
# or 
#CSVJson << print_starting

Overwriting MyCSVDAG.py


In [7]:
import pandas as pd 
def CSVToJson():
    df=pd.read_csv('data/data.csv')
    return df 

df=CSVToJson()

In [8]:
df.head(10)

,name,age,street,city,state,zip,lng,lat
0,Shelley Greer,20,7173 Nicole Parkway Apt. 667,East Keith,Illinois,70445,-36.947689,-23.136564
1,Timothy Lopez,47,746 Todd Expressway,Port Rebeccaborough,Alabama,61799,29.399188,-60.357361
2,Casey Saunders,50,5103 Joseph Freeway,Grantchester,Mississippi,78145,-96.687315,71.719995
3,Connor Merritt,40,0750 Parker Shoal Apt. 487,East Stephenton,Missouri,28716,-158.731271,75.264063
4,James Dixon,77,611 Santiago Prairie Apt. 110,Katieberg,Oregon,19018,-95.679162,-57.370881
5,Patricia Williams,78,5333 Marquez Unions,Lindseychester,Arizona,86581,-56.120500,22.489014
6,Linda Butler,26,2664 Jack Dam,Meredithfort,New Jersey,66087,174.781675,51.381366
7,James Morales,76,7476 Leach Land,New Jacquelinebury,Delaware,51251,10.674868,-32.938641
8,Elizabeth Simpson,69,5494 Stafford Lights Apt. 771,Carlosberg,Arizona,11639,-120.101996,-20.400383
9,Adam Murphy,53,71402 Joseph Underpass Suite 118,Port Tara,South Carolina,14520,174.355155,-15.400470


In [9]:
df.to_json('data/fromAirflow.json',orient='records')